In [1]:
%run ../../src/binning.py
%run ../../src/plot.py
ROOT.EnableImplicitMT(8)

Welcome to JupyROOT 6.22/06


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-5j98m4it because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
#######################################################################
# 2d Rectangular Binning
#######################################################################

rect_bin_factory = RectBinFactory()
rect_bin_factory.make_bins("rec_z",np.array([0.3,0.38,0.5,0.6,0.7,0.9]))
rect_bin_factory.make_bins("rec_pTtot",np.array([0,0.2,0.4,0.6,0.8,1,1.5]))
#rect_bin_factory.make_bins("Mh",np.array([0,0.2,0.4,0.6,0.8,1,1.5]))

In [3]:
#######################################################################
# Custom Binning
#######################################################################

custom_bin_factory = CustomBinFactory(["rec_x","rec_Q2"])

# Add (x,Q2) boundaries
low_Q2 = "rec_Q2>1.4144 + -5.4708 * rec_x + 40.5357 * rec_x*rec_x + -40.0208 * rec_x*rec_x*rec_x + 29.2121 * rec_x*rec_x*rec_x*rec_x"
lmiddle_Q2="rec_Q2<0.6361935324019532 + 5.961630973508846*rec_x + 12.028695097029118*rec_x*rec_x"
hmiddle_Q2="rec_Q2>0.6361935324019532 + 5.961630973508846*rec_x + 12.028695097029118*rec_x*rec_x"
high_Q2 = "rec_Q2<rec_x*17"

custom_bin_factory.add_curve("low_Q2",low_Q2)
custom_bin_factory.add_curve("lmiddle_Q2",lmiddle_Q2)
custom_bin_factory.add_curve("hmiddle_Q2",hmiddle_Q2)
custom_bin_factory.add_curve("high_Q2",high_Q2)

custom_bin_factory.make_bin(["low_Q2","high_Q2"],"rec_x>0.07","rec_x<0.12")
custom_bin_factory.make_bin(["hmiddle_Q2","high_Q2"],"rec_x>0.12","rec_x<0.15")
custom_bin_factory.make_bin(["low_Q2","lmiddle_Q2"],"rec_x>0.12","rec_x<0.2")
custom_bin_factory.make_bin(["hmiddle_Q2","high_Q2"],"rec_x>0.15","rec_x<0.22")
custom_bin_factory.make_bin(["low_Q2","lmiddle_Q2"],"rec_x>0.2","rec_x<0.275")
custom_bin_factory.make_bin(["hmiddle_Q2","high_Q2"],"rec_x>0.22","rec_x<0.29")
custom_bin_factory.make_bin(["low_Q2","lmiddle_Q2"],"rec_x>0.275","rec_x<0.42")
custom_bin_factory.make_bin(["hmiddle_Q2","high_Q2"],"rec_x>0.29","rec_x<0.42")
custom_bin_factory.make_bin(["low_Q2","high_Q2"],"rec_x>0.42","rec_x<1")

In [4]:
#######################################################################
# Load all Binnings into the BinManager
#######################################################################

bin_manager = BinManager()
bin_manager.load_factory(rect_bin_factory)
bin_manager.load_factory(custom_bin_factory)

In [14]:
%run ../../src/plot.py

In [15]:
data_files=get_first_N_files("../../projects/test_data/volatile/data/piplus_piminus/",25,"nSidis_RGA_*")
mc_files=get_first_N_files("../../projects/test/volatile/data/piplus_piminus/",10,"MC_RGA*")
data_tree=ROOT.TChain("dihadron_cuts")
mc_tree=ROOT.TChain("dihadron_cuts")
for data_file in data_files:
    data_tree.Add(data_file)
for mc_file in mc_files:
    mc_tree.Add(mc_file)

In [16]:
histograms=make_multiple_bin_histos(bin_manager, data_tree, mc_tree, 
                         [(";x;Q^{2} [GeV^{2}]",100,0,0.8,100,0,12),
                         (";z;p_{T} [GeV]",100,0.25,1,100,0,1.5),
                         (";M_{h} [GeV];Counts",100,0,2.5)],
                                    plot_variable="rec_Mh")

In [ ]:
canvas = plot_map(bin_manager,histograms,data_tree,mc_tree,64)
canvas.Draw()